<a href="https://colab.research.google.com/github/Carapet07/Data-Science-projects/blob/main/First_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##My first NLP project

In [ ]:
import os, pathlib, shutil, random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  1703k      0  0:00:48  0:00:48 --:--:-- 2141k
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [ ]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

# Move 20% of the training data into new validation directory
for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files) # shuffle files for randomness
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]

  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [ ]:
batch_size = 32

# Create datasets from directories, automatically batching text samples
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [ ]:
for text_batch, label_batch in train_ds.take(1):  # Take one batch
    for i in range(2):  # Show the first 5 examples
        print(f"Text: {text_batch.numpy()[i].decode('utf-8')}")
        print(f"Label: {label_batch.numpy()[i]}")
        print("=" * 50)


Text: Genghis Cohn is a (very) mildly entertaining British movie about a German police commissioner in the late 1950's who is haunted by the ghost of a Jewish comedian that he killed 15 years earlier while serving under Hitler in the SS. The ghost comes back and wants his killer to live as a Jew to atone for the murders he committed.<br /><br />Otto, the German policeman actually knows this ghost's name because, the last thing he did before he died was said, in Yiddish, `Kiss my ass'. The policeman didn't speak Yiddish, so he asked around until he found the meaning. The `kiss my ass' left such an impression that everybody involved with that killing learned and remembered the comedian's name, Genghis Cohn.<br /><br />There are a bunch of men who are murdered in the jurisdiction of the police commissioner, and there are no helpful clues. The men are murdered with a set of knives that are missing from the local butcher. The butcher announces that his knives are missing while the commissio

In [ ]:
max_length = 600 # max length of a movie review
max_tokens = 20000 # max number of unique words to consider

# Create a TextVectorization layer that converts text into integer sequences
# eg: "dog": [0.2, 0.5, -0.7, 0.21... embedding_dim]
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_length
)

# Extract only the text fron train_ds, removing lables (0, 1)
text_only_train_ds = train_ds.map(lambda x, y: x)

# This line adapts the TextVectorization layer to the dataset, learning the vocabulary
# by assigning each word in the dataset a unique integer index
text_vectorization.adapt(text_only_train_ds)

# apply it to every dataset we have
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)

In [ ]:
for inputs, targets in train_ds:
  print("input shape: ", inputs.shape)
  print("input dtype: ", inputs.dtype)
  print("targets shape: ", targets.shape)
  print("targets dtype: ", targets.dtype)
  print('inputs[0]: ', inputs[0])
  print('targets[0]: ', targets[0])
  break

input shape:  (32,)
input dtype:  <dtype: 'string'>
targets shape:  (32,)
targets dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor(b"There is no doubt that this film has an impressive cast but unfortunately this doesn't help with the major downsides to the movie. I never understand why directors ask actors/actresses to use accents not their own when it is obvious to everyone they can't convince. Fiennes just can't do Irish and Fitzgerald isn't much better at Russian. When the voice is wrong then no matter how good the acting the character will never be convincing. As the for the major problem....the plot....was there one? I guess there was some sort of storyline involved but it was so full of holes that I just couldn't wait for the film to end...it was ridiculous. Save 90 minutes of your life and don't watch this movie!", shape=(), dtype=string)
targets[0]:  tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
# downlaod GloVe precomputed database for words ebedding task
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-02-16 06:40:49--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-02-16 06:40:49--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-02-16 06:40:50--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

Parse the unzipped file (.txt file) to build an index that maps words(as strings) to their words reprezentation

In [ ]:
glove_100d_path = "glove.6B.100d.txt"
# 1: Create an embedding dictionary
embeddings_index = {}

# 2: Parse the GloVe txt file. Add each word and it's embedding into emeddings_index dictionary
with open(glove_100d_path) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ") # converts coefs strings into float32
    embeddings_index[word] = coefs

print(f'found {len(embeddings_index)} words')

found 400000 words


Next lets build an embedding matrix that will be loaded into an Embedding layer

In [ ]:
embedding_dim = 100

# 1: Get the vocabulary from the TextVectorizaton layer
vocabulary = text_vectorization.get_vocabulary()

# 2: Assign to each word a uniquie number
word_index = dict(zip(vocabulary, range(len(vocabulary))))

# 3: Create a matrix filled with zeros
embedding_matrix = np.zeros((max_tokens, embedding_dim))

# 4: Populate embedding_matrxi with GloVe embeddinngs for each word in the vocabulary
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector


In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True
)

This dataset yields inputs that are tf.string tensors and targets that are int32 tensors encoding the value "0" or "1"


All set, now let's try learnign something from this data


In [ ]:
inputs = keras.Input(shape=(None, ), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                      save_best_only=True)
    ]

history = model.fit(int_train_ds.cache(),
  validation_data=int_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")